# Generating counterfactuals for multi-class classification and regression models
This notebook will demonstrate how the DiCE library can be used for multiclass classification and regression for scikit-learn models. For demonstration, we will be using the genetic algorithm for CFs.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import dice_ml
from dice_ml import Dice

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd

We will use sklearn's internal datasets to demonstrate DiCE's features in this notebook

In [ ]:
outcome_name = 'target'
# Function to process sklearn's internal datasets
def sklearn_to_df(sklearn_dataset):
    df = pd.DataFrame(sklearn_dataset.data, columns=sklearn_dataset.feature_names)
    df[outcome_name] = pd.Series(sklearn_dataset.target)
    return df

## Multiclass Classification

For multiclass classification, we will use sklearn's Iris dataset. This data set consists of 3 different types of irises’ (Setosa, Versicolour, and Virginica) petal and sepal length. More information at https://scikit-learn.org/stable/datasets/toy_dataset.html#iris-plants-dataset

In [ ]:
from sklearn.datasets import load_iris
df_iris = sklearn_to_df(load_iris())
df_iris.head()

In [ ]:
df_iris.info()

In [ ]:
continuous_features_iris = df_iris.drop(outcome_name, axis=1).columns.tolist()
target = df_iris[outcome_name]

In [ ]:
# Split data into train and test
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier

datasetX = df_iris.drop(outcome_name, axis=1)
x_train, x_test, y_train, y_test = train_test_split(datasetX, 
                                                    target, 
                                                    test_size = 0.2,
                                                    random_state=0,
                                                    stratify=target)

categorical_features = x_train.columns.difference(continuous_features_iris) 

# We create the preprocessing pipelines for both numeric and categorical data.
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformations = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, continuous_features_iris),
        ('cat', categorical_transformer, categorical_features)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf_iris = Pipeline(steps=[('preprocessor', transformations),
                      ('classifier', RandomForestClassifier())])
model_iris = clf_iris.fit(x_train, y_train)

In [ ]:
d_iris = dice_ml.Data(dataframe=df_iris, 
                      continuous_features=continuous_features_iris, 
                      outcome_name=outcome_name)

# We provide the type of model as a parameter (model_type)
m_iris = dice_ml.Model(model=model_iris, backend="sklearn", model_type='classifier')

In [ ]:
exp_genetic_iris = Dice(d_iris, m_iris, method="random")

As we can see below, all the target values will lie in the desired class

In [ ]:
# Single input 
query_instances_iris = x_train[2:3]
genetic_iris = exp_genetic_iris.generate_counterfactuals(query_instances_iris, total_CFs=4, desired_class = 2, posthoc_sparsity_param=None)
genetic_iris.visualize_as_dataframe()

In [ ]:
# Multiple queries can be given as input at once
query_instances_iris = x_train[17:19]
genetic_iris = exp_genetic_iris.generate_counterfactuals(query_instances_iris, total_CFs=7, desired_class = 2, posthoc_sparsity_param=None)
genetic_iris.visualize_as_dataframe(show_only_changes=True)

# Regression

For regression, we will use sklearn's boston dataset. This dataset contains boston house-prices. More information at https://scikit-learn.org/stable/datasets/toy_dataset.html#boston-house-prices-dataset

In [ ]:
from sklearn.datasets import load_boston
df_boston = sklearn_to_df(load_boston())
df_boston.head()

In [ ]:
df_boston.info()

In [ ]:
continuous_features_boston = df_boston.drop(outcome_name, axis=1).columns.tolist()
target = df_boston[outcome_name]

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
# Split data into train and test
datasetX = df_boston.drop(outcome_name, axis=1)
x_train, x_test, y_train, y_test = train_test_split(datasetX, 
                                                    target, 
                                                    test_size = 0.2,
                                                    random_state=0)

categorical_features = x_train.columns.difference(continuous_features_boston) 

# We create the preprocessing pipelines for both numeric and categorical data.
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformations = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, continuous_features_boston),
        ('cat', categorical_transformer, categorical_features)])

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
regr_boston = Pipeline(steps=[('preprocessor', transformations),
                      ('regressor', RandomForestRegressor())])
model_boston = regr_boston.fit(x_train, y_train)

In [ ]:
d_boston = dice_ml.Data(dataframe=df_boston, continuous_features=continuous_features_boston, outcome_name=outcome_name)
# We provide the type of model as a parameter (model_type)
m_boston = dice_ml.Model(model=model_boston, backend="sklearn", model_type='regressor')

In [ ]:
exp_genetic_boston = Dice(d_boston, m_boston, method="random")

As we can see below, all the target values will lie in the desired range

In [ ]:
# Multiple queries can be given as input at once
query_instances_boston = x_train[2:3]
genetic_boston = exp_genetic_boston.generate_counterfactuals(query_instances_boston, 
                                                             total_CFs=2, 
                                                             desired_range=[30, 45])
genetic_boston.visualize_as_dataframe(show_only_changes=True)

In [ ]:
# Multiple queries can be given as input at once
query_instances_boston = x_train[17:19]
genetic_boston = exp_genetic_boston.generate_counterfactuals(query_instances_boston, total_CFs=4, desired_range=[40, 50])
genetic_boston.visualize_as_dataframe(show_only_changes=True)